<a href="https://colab.research.google.com/github/mayhd3/NSF-REU-2021/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if not os.path.exists('Xtst.csv'):
  !gdown --id 1wQDp91LC0PBJvJRBTLw1aW51-UN5JOm8
if not os.path.exists('Ytr.csv'):
  !gdown --id 1ZhPVjXlOttrYRjSRm_IUBZj-Bokswzbn
if not os.path.exists('Ytst.csv'):
  !gdown --id 10LE7A7yhNkPlk8Zv3dmBjFYwMW9ZKy9_
if not os.path.exists('Xtr.csv'):
  !gdown --id 1IUcv_OcLX1YKUB1Xw_lYiBK6bnyLxikS
if not os.path.exists('cnn1d.zip'):
  !curl --remote-name -H 'Accept: application/vnd.github.v3.raw' --location 'https://github.com/mayhd3/NSF-REU-2021/raw/main/cnn1d.zip'

In [ ]:
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Xtr = pd.read_csv("Xtr.csv", sep=',', header=None)
Xtst = pd.read_csv("Xtst.csv", sep=',', header=None)
Ytr = pd.read_csv("Ytr.csv", sep=',', header=None)
Ytst = pd.read_csv("Ytst.csv", sep=',', header=None)

scaler = preprocessing.StandardScaler().fit(Xtr)
xtr = scaler.transform(Xtr)  
xtst = scaler.transform(Xtst)  

xtr = xtr.reshape(xtr.shape[0],xtr.shape[1],1)
xtst = xtst.reshape(xtst.shape[0],xtst.shape[1],1)

print ("Number of training samples is: ",xtr.shape[0])
print ("Number of test samples is: ",xtst.shape[0])

num_classes = 2

ytr = to_categorical(Ytr, num_classes)
ytst = to_categorical(Ytst, num_classes)

model = None
if not os.path.exists('cnn1d.zip'):
  model = Sequential()
  model.add(Conv1D(32, kernel_size=7, activation='relu', input_shape=xtr.shape[1:]))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))    
  model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
  model.fit(xtr, ytr, epochs=150, verbose=2, batch_size=128,validation_split=0.3)  #learning_rte
  model.save('cnn1d')
  !zip -r cnn1d.zip cnn1d

else:
  if not os.path.exists('cnn1d'):
    !unzip cnn1d.zip
  model = load_model('cnn1d')

Number of training samples is:  1029085
Number of test samples is:  256883


In [ ]:
print("Evaluation of best performing model:")
model.evaluate(xtst, ytst)

Evaluation of best performing model:
8028/8028 [==============================] - 27s 2ms/step - loss: 0.7905 - accuracy: 0.9367


[0.7904874682426453, 0.9367416501045227]

In [ ]:
import numpy as np

def percentiles(x,y,p):
  labels = np.argmax(y, axis=1)
  totals = [
    np.sum( np.compress(labels == label, x, axis=0), axis=1 ).flatten()
      for label in range(labels.max()+1)
  ]
  print([len(total) for total in totals])
  return [np.percentile(total, p) for total in totals]

print(percentiles(xtr,ytr,10))


[514285, 514800]
[-20.752298179341157, -27.510753166929373]


In [ ]:
try:
  from art.attacks.evasion import FastGradientMethod, Wasserstein
  from art.estimators.classification import TensorFlowV2Classifier
except ImportError:
  !{sys.executable} -m pip install adversarial-robustness-toolbox[Keras]
import tensorflow as tf

# Train the ART classifier

classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=xtr.shape,
    loss_object=tf.keras.losses.CategoricalCrossentropy(
      from_logits=False,
#      label_smoothing=0,
#      axis=-1,
#      reduction="auto",
      name="categorical_crossentropy"
  )
)
predictions = classifier.predict(xtst)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

# Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.2)
xtst_adv = attack.generate(x=xtst)

# Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(xtst_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(ytst, axis=1)) / len(ytst)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 93.67416294577687%
Accuracy on adversarial test examples: 66.14684506175963%
